In [1]:
import pandas as pd
import numpy as np
import magic
from matplotlib import pyplot as plt
import importlib
%matplotlib inline
importlib.reload(magic)
import seaborn as sns

In [23]:
data = pd.read_csv('exp.csv')
data['tournament'] = data.tournament_scores/data.num_players
data['effectiveness'] = np.log(data.scores+1)/np.log(data.map_rivers_count/data.num_players)


In [26]:
data = data.groupby('MineWeight').mean().loc[:,['effectiveness','effectiveness']].reset_index()


In [28]:
plt.scatter(data.MineWeight,data.tournament)

AttributeError: 'DataFrame' object has no attribute 'tournament'